Due to conflicts, ran in google CoLab in class

In [1]:
from tensorflow.keras.datasets import mnist

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
y_test.shape

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
sns.set(font_scale=2)

In [ ]:
import numpy as np
index = numpy.random.choice(np.arange(len(X_train)), 24, replace=False)
figure, axes = plt.subplots(nrows=4, ncols=6, figsize=(16, 9))

for item in zip(axes.ravel(), X_train[index], y_train[index]):
    axes, image, target = item
    axes.imshow(image, cmap=plt.cm.gray_r)
    axes.set_xticks([])
    axes.set_yticks([])
    axes.set_title(target)

plt.tight_layout()

Data Prep

In [ ]:
X_train = X_train.reshape(60000, 28, 28, 1)
X_train.shape

In [ ]:
X_test = X_test.reshape(10000, 28, 28, 1)
X_test.shape

Normalising the image data

In [ ]:
X_train = X_train.astype('float32')/255
X_test = X_test.astype('float32')/255

One-Hot Encoding

In [ ]:
from tensorflow.keras.utils import to_categorical

y_train = to_categorical(y_train)
y_train.shape

In [ ]:
from tensorflow.keras.models import Sequential

cnn = Sequential()

In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

cnn.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))

In [ ]:
cnn.add(MaxPooling2D(pool_size=(2, 2)))
cnn.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu'))
cnn.add(MaxPooling2D(pool_size=(2, 2)))
cnn.add(Flatten())
cnn.add(Dense(units=128, activation='relu'))
cnn.add(Dense(units=10, activation='softmax'))
cnn.summary()

In [ ]:
from tensorflow.keras.utils import plot_model
from IPython.display import Image

plot_model(cnn, to_file='convnet.png', show_shapes=True, show_layer_names=True)
Image(filename='convnet.png')

In [ ]:
cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
cnn.fit(X_train, y_train, epochs=5, validation_split=0.1)

In [ ]:
loss, accuracy = cnn.evaluate(X_test, y_test)

In [ ]:
loss

In [ ]:
accuracy

In [ ]:
predictions = cnn.predict(X_test)
y_test[0]

In [ ]:
for index, probability in enumerate(predictions[0]):
    print(f'{index}: {probability:.10%}')

In [ ]:
images = X_test.reshape(10000, 28, 28)
incorrect_predictions = []

for i, (p, e) in enumerate(zip(predictions, y_test)):
    predicted, expected = np.argmax(p), np.argmax(e)

    if predicted != expected:
        incorrect_predictions.append((i, images[i], predicted, expected))

In [ ]:
len(incorrect_predictions)

Visualising incorrect predictions

In [ ]:
figure, axes = plt.subplots(nrows=4, ncols=6, figsize=(16, 12))

for axes, item in zip(axes.ravel(), incorrect_predictions):
    index, image, predicted, expected = item
    axes.imshow(image, cmap=plt.cm.gray_r)
    axes.set_xticks([])
    axes.set_yticks([])
    axes.set_title(f'index: {index}  predicted: {predicted}  expected: {expected}')

plt.tight_layout()

In [ ]:
def display_probabilities(prediction):
    for index, probability in enumerate(prediction):
        print(f'{index}: {probability:.10%}')
    

In [ ]:
display_probabilities(predictions[495])

In [ ]:
display_probabilities(predictions[583])

In [ ]:
from tensorflow.keras.models import load_model
cnn = load_model('saved_model/mnist_cnn.h5')
cnn.summary()